In [5]:
import boto3

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
role = 'arn:aws:iam::057055853158:role/SageMake_Test'

from time import gmtime, strftime

model_name = 'bai-pet-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://cycle-gan-model-zoo/model.tar.gz'
container = '{}.dkr.ecr.{}.amazonaws.com/bai-pet-v5:latest'.format(account_id, region)
instance_type = 'ml.t2.medium'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
'Image': container,
'ModelDataUrl': model_url,
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = container)

print("Model Arn: " + create_model_response['ModelArn'])


endpoint_config_name = 'bai-pet-config-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])
        
print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

endpoint_name = 'bai-pet-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Model name: bai-pet-2025-05-06-09-47-30
Model data Url: s3://cycle-gan-model-zoo/model.tar.gz
Container image: 057055853158.dkr.ecr.us-west-2.amazonaws.com/bai-pet-v5:latest
Model Arn: arn:aws:sagemaker:us-west-2:057055853158:model/bai-pet-2025-05-06-09-47-30
Endpoint config name: bai-pet-config-2025-05-06-09-47-31
Endpoint config Arn: arn:aws:sagemaker:us-west-2:057055853158:endpoint-config/bai-pet-config-2025-05-06-09-47-31
Endpoint name: bai-pet-endpoint-2025-05-06-09-47-31
Endpoint Arn: arn:aws:sagemaker:us-west-2:057055853158:endpoint/bai-pet-endpoint-2025-05-06-09-47-31
Endpoint Status: Creating
Waiting for bai-pet-endpoint-2025-05-06-09-47-31 endpoint to be in service...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:62                                                                                   │
│                                                                                                  │
│   59                                                                                             │
│   60 print('Waiting for {} endpoint to be in service...'.format(endpoint_name))                  │
│   61 waiter = sm_client.get_waiter('endpoint_in_service')                                        │
│ ❱ 62 waiter.wait(EndpointName=endpoint_name)                                                     │
│   63                                                                                             │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/waiter.py:58 in wait    │
│                                                                                                  │
│    55 │   # Waiter.wait method. This is needed to attach a docstring to the                      │
│    56 │   # method.                                                                              │
│    57 │   def wait(self, **kwargs):                                                              │
│ ❱  58 │   │   Waiter.wait(self, **kwargs)                                                        │
│    59 │                                                                                          │
│    60 │   wait.__doc__ = WaiterDocstring(                                                        │
│    61 │   │   waiter_name=waiter_name,                                                           │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/context.py:123 in       │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/waiter.py:396 in wait   │
│                                                                                                  │
│   393 │   │   │   │   │   reason=reason,                                                         │
│   394 │   │   │   │   │   last_response=response,                                                │
│   395 │   │   │   │   )                                                                          │
│ ❱ 396 │   │   │   time.sleep(sleep_amount)                                                       │
│   397                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [6]:
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
print("Status:", response["EndpointStatus"])
print("Failure reason:", response.get("FailureReason", "No reason provided"))


Status: Creating
Failure reason: No reason provided


In [7]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sm_client.delete_endpoint(EndpointName=endpoint_name)                                        │
│   2 sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)                    │
│   3 sm_client.delete_model(ModelName=model_name)                                                 │
│   4                                                                                              │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/client.py:570 in        │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/context.py:123 in       │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /Users/yche14/anaconda3/envs/e2eml/lib/python3.10/site-packages/botocore/client.py:1031 in       │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                                                             │
│   1030 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1031 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1032 │   │   else:                                                                             │
│   1033 │   │   │   return parsed_response                                                        │
│   1034                                                     